In [1]:
import pandas as pd

In [37]:
data = pd.read_csv('merged_cleaned_validation.csv', delimiter = ',')
tester = data['lyrics_cleaned'][:50]

In [15]:
from git import Repo
from pathlib import Path

Repo.clone_from('https://github.com/JULIELab/X-ANEW.git', Path.home()/'Documents/GitHub/CogSci2-Spotify/anew')

GitCommandError: Cmd('git') failed due to: exit code(128)
  cmdline: git clone -v https://github.com/JULIELab/X-ANEW.git /Users/yananikolova/Documents/GitHub/CogSci2-Spotify/anew
  stderr: 'fatal: destination path '/Users/yananikolova/Documents/GitHub/CogSci2-Spotify/anew' already exists and is not an empty directory.
'

In [33]:
xanew = pd.read_csv('anew/Ratings_Warriner_et_al.csv', delimiter = ',')

In [35]:
xanew = xanew[['Word', 'V.Mean.Sum', 'A.Mean.Sum', 'D.Mean.Sum']]
xanew.columns = ['word', 'valence', 'arousal', 'dominance']
xanew.set_index('word', inplace=True)
xanew.head()

,valence,arousal,dominance
word,,,
aardvark,6.26,2.41,4.27
abalone,5.30,2.65,4.95
abandon,2.84,3.73,3.32
abandonment,2.63,4.95,2.64
abbey,5.85,2.20,5.00


In [49]:
from nltk.stem import WordNetLemmatizer
 
lemmatizer = WordNetLemmatizer()

In [52]:
import nltk as nltk
nltk.download('omw-1.4')

[nltk_data] Downloading package omw-1.4 to
[nltk_data]     /Users/yananikolova/nltk_data...
[nltk_data]   Unzipping corpora/omw-1.4.zip.


True

In [102]:
tokens = set(lemmatizer.lemmatize(word.lower()) for word in tester.iloc[22].split(' '))

In [103]:
affective_words = list(set(xanew.index).intersection(tokens))
print(affective_words)

['secret', 'new', 'running', 'silhouette', 'constellation', 'attack', 'hell', 'prison', 'protection', 'zone', 'begin', 'pit', 'blood', 'chain', 'can', 'light', 'palomino', 'back', 'season', 'red', 'see', 'whip', 'cherry', 'trend', 'consolation', 'zip', 'blond', 'right', 'girl', 'envy', 'notice', 'familiar', 'feminist', 'woman', 'good', 'whisper', 'eye', 'shell', 'look', 'carpet', 'urban', 'get']


In [106]:


xanew.loc[affective_words]

,valence,arousal,dominance
word,,,
secret,5.33,4.14,5.41
new,7.68,5.14,5.22
running,6.53,5.52,5.89
silhouette,6.10,3.14,5.20
constellation,6.24,4.35,5.14
attack,2.00,7.05,3.39
hell,2.55,6.26,3.47
prison,1.94,5.10,3.54
protection,6.73,4.32,6.71


In [76]:
!pip install vaderSentiment

In [82]:
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
analyzer = SentimentIntensityAnalyzer()

In [105]:
analyzer.polarity_scores(tester.iloc[22])

{'neg': 0.114, 'neu': 0.834, 'pos': 0.052, 'compound': -0.9061}

In [94]:
data[['y_valence', 'y_arousal']][:50]

,y_valence,y_arousal
0,-1.176640,-0.314720
1,-0.780962,-0.789480
2,-0.767318,0.911361
3,-1.487725,-0.360813
4,1.071901,0.846830
5,-1.935250,-0.655810
6,0.815393,0.662457
7,-0.363454,-0.487569
8,1.178325,1.183311
9,-1.935250,-0.655810
